<a href="https://colab.research.google.com/github/s-c-soma/AlternusVera/blob/master/TheMeanSquares_AlternusVera/News%20Coverage_Subarna/Factor_NewsCoverage_ScoreComputing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Part Two - Analysing related article sentiment and bias
This workbook considers the second part of the project - taking a group of related articles and assessing their sentiment and scope for bias.
In an ideal world, where we each want to have a fair and balanced view on every topic, we would hope to read articles covering a range of perspectives on the same story.
Having developed the means to find related articles in part one of this project, this part applies several sentiment analysis techniques to look at the distribution of their sentiment. It ultimately arrives at a score to convey how balanced the coverage of a story is.

## Preparation
### Imports
First some of the required packages must be imported.

### Import Library

In [ ]:
import argparse
import pandas as pd
import numpy as np
import operator
import matplotlib.pyplot as plt
import nltk as nl
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import ParameterGrid
import statistics
import random
import warnings
import re
from matplotlib import pyplot
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB



/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### Parameter configuration
The parameters used to control the NLP-related calculations, and to specify the domain for any grid search are captured in the runParams dict. This includes specification of the location of the key input files.
The runParams dict is converted into an sklearn ParameterGrid, even if there is no grid search requirement (in which case it's processed as a single scenario grid search).

In [ ]:
#'google','vader','stanford'
runParams={'sentiment_library':   ['vader'],
           'input_file':          ['/content/drive/My Drive/Colab Notebooks/articles1.csv'],
           'output_file':         ['/content/drive/My Drive/Colab Notebooks/fakenews_by_news-coverage.csv'],
           'article_id_list':     [[120639,80103,25225,21502,57362,120636]],
           'sentiment_sentences': [5],
           'article_stats':       [False]}

# Use parameter grid even if there is only set of parameters
parameterGrid=ParameterGrid(runParams)

In [ ]:
#financial_stock_data =  pd.read_csv('/content/drive/My Drive/Colab Notebooks/articles1.csv', low_memory =False)
#financial_stock_data.shape

### File loader for news corpus

This is the same function as used in Part One.

In [ ]:
def getInputDataAndDisplayStats(filename,processDate,printSummary=False):

	df=pd.read_csv(filename)

	df=df.drop_duplicates('content')
	df=df[~df['content'].isnull()]

	
	df=df[df['content'].str.len()>=200]

	# Find and remove summary NYT "briefing" articles to avoid confusing the clustering
	targetString="(Want to get this briefing by email?"
	df['NYT summary']=df['content'].map(lambda d: d[:len(targetString)]==targetString)
	df=df[df['NYT summary']==False]

	# The following removes a warning that appears in many of the Atlantic articles.
	# Since it is commonly at the beginning, it brings a lot of noise to the search for similar articles
	# And subsequently to the assessment of sentiment
	targetString="For us to continue writing great stories, we need to display ads.             Please select the extension that is blocking ads.     Please follow the steps below"
	df['content']=df['content'].str.replace(targetString,'')

	# This is also for some Atlantic articles for the same reasons as above
	targetString="This article is part of a feature we also send out via email as The Atlantic Daily, a newsletter with stories, ideas, and images from The Atlantic, written specially for subscribers. To sign up, please enter your email address in the field provided here."
	df=df[df['content'].str.contains(targetString)==False]

	# This is also for some Atlantic articles for the same reasons as above
	targetString="This article is part of a feature we also send out via email as Politics  Policy Daily, a daily roundup of events and ideas in American politics written specially for newsletter subscribers. To sign up, please enter your email address in the field provided here."
	df=df[df['content'].str.contains(targetString)==False]

	# More Atlantic-specific removals (for daily summaries with multiple stories contained)
	df=df[df['content'].str.contains("To sign up, please enter your email address in the field")==False]

	# Remove daily CNN summary
	targetString="CNN Student News"
	df=df[df['content'].str.contains(targetString)==False]

	if printSummary:
		print("\nArticle counts by publisher:")
		print(df['publication'].value_counts())

		print("\nArticle counts by date:")
		print(df['date'].value_counts())
		
	# Restrict to articles on the provided input date.
	# This date is considered mandatory for topic clustering but is not required for sentiment
	# since sentiment only processes a specified list of articles.
	# For topic clustering it is essential to have the date as it is
	# enormously significant in article matching.
	if processDate!=None:
		df=df[df['date']==processDate]
	df.reset_index(inplace=True, drop=True)

	# Remove non-ASCII characters
	df['processed_content']=df['content'].map(lambda x: removeNonASCIICharacters(x))
	df['score'] = 0.0

	print("\nFinal dataset:\n\nDate:",processDate,"\n")
	print(df['publication'].value_counts())

	return df

##########################################################################################

def removeNonASCIICharacters(textString): 
    return "".join(i for i in textString if ord(i)<128)

### Load the articles from the corpus
In addition the function will return the number of articles per publication (for the requested run date). Here we see there is a relatively good mix of political viewpoints covered. More discussion of this is provided in the project report.

In [ ]:
articleDataFrame=getInputDataAndDisplayStats(runParams['input_file'][0],None,False)


Final dataset:

Date: None 

Breitbart           23585
CNN                 11249
New York Times       7620
Business Insider     6504
Atlantic              157
Name: publication, dtype: int64


In [ ]:
articleDataFrame.date.unique

<bound method Series.unique of 0        2016-12-31
1        2017-06-19
2        2017-01-06
3        2017-04-10
4        2017-01-02
            ...    
49110    2017-01-11
49111    2017-01-11
49112    2017-01-11
49113    2017-01-11
49114    2017-01-11
Name: date, Length: 49115, dtype: object>

In [ ]:
articleDataFrame.shape

(49115, 13)

In [ ]:
col_id_list = articleDataFrame['id'].tolist()
#col_id_list 


### Take All Columns

In [ ]:
 articleDataFrame['id'].count()


49115

Comment this line if not want to take all the IDs

In [ ]:
##runParams['article_id_list'][0] = col_id_list 

## News Coverage computation methods
These are to determine a score for how well a set of documents manages to cover a variety of perspectives...

### NC-Score: Sentiment analysis
Three NLP libraries are supported for this part of the project: Vader, Google, and Stanford Core NLP. Classes will be defined to enable each of them.
### Parent Sentiment Analysis Class
The main sentiment analyser class provides a consistent wrapper and interface around classes specific to the various NLP libraries. Its constructor creates and embeds an instance of the appropriate class. It provides additional standard interfaces to trigger the analysis on an article, and to return the overall score for that article.
Additionally it manages the scaling of results across classes in order to facilitate consistency.

In [ ]:
class SentimentAnalyser():

	scaleMin=-1.
	scaleMax=1.

    # Initializer / Instance attributes
	def __init__(self, library):
		if library=='vader':
			self.analyser=NLTKVaderSentimentAnalyser()
		else:
			print("ERROR - NO RECOGNISED LIBRARY")

	def getOverallArticleScore(self,articleResults):

		# Google returns a document score, but it is an int, which is useful when comparing documents
		# Hence computing the average of the sentences here instead
		# Google's document score is here: articleResults.document_sentiment.score
		numSentences=0.
		totalSentScore=0.
		for sentence in articleResults:
			numSentences+=1
			totalSentScore+=self.analyser.getSentenceScoreFromResults(sentence)

		value=(totalSentScore/numSentences-self.analyser.scaleMin)/(self.analyser.scaleMax-self.analyser.scaleMin)
		return self.scaleMin+value*(self.scaleMax-self.scaleMin)

	def generateResults(self,textToAnalyse):
		return self.analyser.generateResults(textToAnalyse)

### Vader library class
This class does the same thing for using the VADER sentiment analyser packaged with NLTK.

In [ ]:
class NLTKVaderSentimentAnalyser():
# Per NLTK Vader user guide: https://pypi.org/project/vaderSentiment/
# Typical threshold values (used in the literature cited on this page) are: 
#. **positive sentiment**: ``compound`` score >= 0.05 
#. **neutral sentiment**: (``compound`` score > -0.05) and (``compound`` score < 0.05) 
#. **negative sentiment**: ``compound`` score <= -0.05 

	scaleMin=-1.
	scaleMax=1.

	def __init__(self):
		self.nltkVaderAnalyser=SentimentIntensityAnalyzer()
		return

	def generateResults(self,textToAnalyse):
		ss=[]
		for sentence in nl.sent_tokenize(textToAnalyse):
			ss.append(self.nltkVaderAnalyser.polarity_scores(sentence))
		return ss

	def getSentenceScoreFromResults(self,sentenceResults):
		return sentenceResults['compound']

### Computing Balance Score

In [ ]:
def computePopulationBalanceScore(articleScoreDict,sentimentClass):
	# Extract values from dict, then normalise to be within -1 to +1
	# Then compute population standard deviation as the balance score
	population=[-1.+(x-sentimentClass.scaleMin)/(sentimentClass.scaleMax-sentimentClass.scaleMin)*(1.-(-1.)) for x in articleScoreDict.values()]
	return statistics.pstdev(population)



### Overall- Avg Score Computation for Story

In [ ]:
def computePopulationBalanceScoreHistoMean(articleScoreDict,sentimentClass):
	# Extract values from dict, then normalise to be within -1 to +1
	# Then compute population standard deviation as part of the balance score
	numBuckets=len(articleScoreDict)
	articleValues=pd.Series(articleScoreDict)
	
	# Based on 10,000 random article samples, Google's sentiment score for these articles lies within +/- 0.86
	# So, scale all scores by dividing by that value to rescale to +/- 1.00 before computing balance score
	# Ideally this should factored in at the individual NLP library class level 
	articleValues=articleValues/0.86

	populatedBuckets=0
	for i in range(numBuckets):
		bucketFrom=sentimentClass.scaleMin+i*(sentimentClass.scaleMax-sentimentClass.scaleMin)/numBuckets
		bucketTo=bucketFrom+(sentimentClass.scaleMax-sentimentClass.scaleMin)/numBuckets
		# The following is to ensure the top of the highest bucket is counted somewhere
		# and doesn't fall out due to treatment of inequalities in ranges
		if bucketTo==sentimentClass.scaleMax:
			bucketTo+=0.001
		numSamples=((bucketFrom<=articleValues) & (articleValues<bucketTo)).sum()
		if numSamples>0:
			populatedBuckets+=1

	# Score computed as proportion of buckets which are populated (more buckets implies a more balanced view)
	# This has a value between 0 and 1.
	# This is in turn multiplied by the distance between the mean and 1.
	# So, if mean is in center (i.e. at 0) then things are balanced, so score is not decreased
	# Otherwise, score is decreased proportionately
	return (populatedBuckets/numBuckets * (1.-abs(articleValues.mean())))

### NC-Score: News Source Credibility 

## Story map loading
The story map file is an optional way to provide the algorithm with:
- A list of stories
- Each story has a name (for reference/convenience)
- Each story contains a list of articles that pertain to that story

The objective is ultimately to process each story, and within each story to measure the sentiment of each article, then (still within the story) to compute the score for the balance of the coverage (of that story).

This section of code is taken from the other workbook.

In [ ]:
def setupStoryMapAndReportList(args=None,reportArticleList=None,storyMapFileName=None):
	# Story Map is used in fitting if grid search is applied (As ground truth)
	# It is also used in graph if no threshold provided (to determine colours, not to determine location)
	# Report Article List is used at the end to create a report with, for each
	# article in the list, the set of articles within tolerance, and the key words for each
	if args==None:
		articleList=reportArticleList
		fileName=storyMapFileName
	else:
		articleList=args['article_id_list']
		fileName=args['story_map_validation']

	reportArticleList=articleList
	if fileName!=None:
		storyMap=readStoryMapFromFile(fileName)
		if reportArticleList==None:
			reportArticleList=[]
			for story, articleList in storyMap.items():
				reportArticleList.append(articleList[0])
	else:
		storyMap=None
	return storyMap,reportArticleList

def readStoryMapFromFile(filename):
	return readDictFromCsvFile(filename,'StoryMap')

##########################################################################################

def readGridParameterRangeFromFile(filename):
	return readDictFromCsvFile(filename,'GridParameters')

##########################################################################################

def readDictFromCsvFile(filename,schema):
	gridParamDict={}
	with open(filename, 'r') as f:
		for row in f:
			row=row[:-1] # Exclude the carriage return
			row=row.split(",")
			key=row[0]
			vals=row[1:]
			
			if schema=='GridParameters':
				if key in ['story_threshold','tfidf_maxdf']:
					finalVals=list(float(n) for n in vals)
				elif key in ['ngram_max','tfidf_mindf','max_length']:
					finalVals=list(int(n) for n in vals)
				elif key in ['lemma_conversion','tfidf_binary']:
					finalVals=list(str2bool(n) for n in vals)
				elif key in ['parts_of_speech']:
					listlist=[]
					for v in vals:
						listlist.append(v.split("+"))
					finalVals=listlist
				elif key in ['tfidf_norm','nlp_library']:
					finalVals=vals
				else:
					print(key)
					print("KEY ERROR")
					return
			elif schema=='StoryMap':
				finalVals=list(int(n) for n in vals)
			else:
				print(schema)
				print("SCHEMA ERROR")
				return
			
			gridParamDict[key]=finalVals
	return gridParamDict

### Load the story map from file

In [ ]:
storyMap,reportArticleList=setupStoryMapAndReportList(storyMapFileName='/content/drive/My Drive/Colab Notebooks/storyMapForValidation1.csv')

In [ ]:
storyMap,reportArticleList=setupStoryMapAndReportList(None,None,None)

Inspecting the story map we see that it forms a dict containing a key corresponding to the name of the story and a value containing a list of the article IDs germane to that story.

In [ ]:
if storyMap is None:
  print('')
else:
  for story, articleList in storyMap.items():
    print(story,":",articleList)

### Augment story map with user requested specific article list
A requested article list is explicitly part of the input parameters and may vary through each iteration of the main loop. This data may not be consistent with any provided story map file. So the following function will be required in order to reconcile any differences.

In [ ]:
def collapseRequestedArticleListIntoStoryList(requestedArticleList,storyMap):
	# Check that the explicitly requested articles are all contained in the storyList
	# If they aren't, add a new story to contain them

	# If the storyMap was empty, it will be None,
	# so initialise as a dictionary ready for adding new values
	if storyMap==None:
		newStoryMap={}
	else:
		newStoryMap=storyMap.copy()

	found=False
	for story,articleListFromMap in newStoryMap.items():
		if len(articleListFromMap)==len(requestedArticleList):
			y=sum([x in articleListFromMap for x in requestedArticleList])
			if y==len(articleListFromMap):
				found=True
                
	# If there is no complete story exactly matching then add a new story to the list
	# With the first article ID as the key (arbitrarily)
	if not found:
		newStoryMap[requestedArticleList[0]]=requestedArticleList
	print('newStoryMap',newStoryMap)
	return newStoryMap

## Main Process Run
Note that the earlier steps need to have been run before this step in order to populate the following:
- articleDataFrame
- parameterGrid



Vader will operate without any additional constraints, providing the Python environment has all the correct packages, per my project requirements file.

In [ ]:
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
for i,currentParams in enumerate(parameterGrid):
  if len(parameterGrid)>1:
    print("Combination:",i+1,"of",len(parameterGrid))
    print(currentParams)

  iterationStoryMap=collapseRequestedArticleListIntoStoryList(currentParams['article_id_list'],storyMap)
  sentimentAnalyser=SentimentAnalyser(currentParams['sentiment_library'])

  for story,articleList in iterationStoryMap.items():
    articleSentScores={}
    print("ANALYSING STORY:",story,"using",currentParams['sentiment_library'])
    print("Number of articles in story:",len(articleList))

    for article in articleList:
      articleContent=articleDataFrame[articleDataFrame['id']==article].iloc[0]['content']
      if currentParams['sentiment_sentences']!=None:
	      articleSentences=nl.sent_tokenize(articleContent)
	      textToAnalyse=' '.join(articleSentences[:currentParams['sentiment_sentences']])	
      else:
	      textToAnalyse=articleContent

      results=sentimentAnalyser.generateResults(textToAnalyse)
      articleSentScores[article]=sentimentAnalyser.getOverallArticleScore(results)
    
    # Sort and display results
    sortedArticleSentScores=sorted(articleSentScores.items(), key=operator.itemgetter(1))
    print("\nArticle sentiments, most positive first:")
    for article in reversed(sortedArticleSentScores):
	      print(article[0],":", round(article[1],3),articleDataFrame[articleDataFrame['id']==article[0]].iloc[0]['publication'])
	      articleDataFrame.at[articleDataFrame[articleDataFrame['id']==article[0]].index[0],'score'] = round(article[1],3)
    

    print("\nNEWS COVERAGE SCORE:",round(computePopulationBalanceScoreHistoMean(articleSentScores,SentimentAnalyser),3),"\n")


newStoryMap {120639: [120639, 80103, 25225, 21502, 57362, 120636]}
ANALYSING STORY: 120639 using vader
Number of articles in story: 6


IndexError: ignored

## Output inspection
To inspect the raw content of one of the articles, substitute the desired ID in the following command and run it.

In [ ]:
articleDataFrame[articleDataFrame['id']==80103]#['content'].values[0]

## Fakeness Label from News Coverage Score

* positive sentiment: ``compound`` score >= 0.05 
* neutral sentiment: (``compound`` score > -0.05) and (``compound`` score < 0.05) 
* negative sentiment: ``compound`` score <= -0.05 

###param try1

In [ ]:
articleDataFrame.loc[articleDataFrame.score >= 0.05, 'label_NewsCoverage'] = 1 # true news
articleDataFrame.loc[((articleDataFrame.score > -0.05) & (articleDataFrame.score < 0.05 )), 'label_NewsCoverage'] = 2 # avg
articleDataFrame.loc[articleDataFrame.score <= -0.05, 'label_NewsCoverage'] = 0 # fake news


###param try2

In [ ]:
articleDataFrame.loc[articleDataFrame.score >= 0.00, 'label_NewsCoverage'] = 1 # true news
#articleDataFrame.loc[((articleDataFrame.score > -0.05) & (articleDataFrame.score < 0.05 )), 'label_NewsCoverage'] = 2 # avg
articleDataFrame.loc[articleDataFrame.score < 0.00, 'label_NewsCoverage'] = 0 # fake news

In [ ]:
print("Size of the Labels column")
print(articleDataFrame.groupby('label_NewsCoverage').size())
articleDataFrame['label_NewsCoverage'].value_counts().plot(kind="bar")
plt.ylabel('label_NewsCoverage')
plt.show()

In [ ]:
articleDataFrame[articleDataFrame.id.isin([120639, 80103, 25225, 21502, 57362, 120636])]

## Classification

### Test and Training Data

In [ ]:
articleDataFrame["label_NewsCoverage"] = articleDataFrame["label_NewsCoverage"].astype('int')

In [ ]:
articleDataFrame.head(2)

In [ ]:
# Separate the dataframe for input(X) and output variables(y)
#['Unnamed: 0', 'id', 'title', 'publication', 'author', 'date', 'year','month', 'url', 'content', 'NYT summary', 'processed_content','score', 'label_NewsCoverage']
X = articleDataFrame['processed_content']
Y = articleDataFrame['label_NewsCoverage']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)
X_train = np.array(X_train);
X_test = np.array(X_test);
Y_train = np.array(Y_train);
Y_test = np.array(Y_test);
cleanHeadlines_train = [] #To append processed headlines
cleanHeadlines_test = [] #To append processed headlines
number_reviews_train = len(X_train) #Calculating the number of reviews
number_reviews_test = len(X_test) #Calculating the number of reviews

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
def get_words( headlines ):               
    headlines_onlyletters = re.sub("[^a-zA-Z]", " ",headlines) #Remove everything other than letters     
    words = headlines_onlyletters.lower().split() #Convert to lower case, split into individual words    
    stops = set(stopwords.words("english"))  #Convert the stopwords to a set for improvised performance                 
    meaningful_words = [w for w in words if not w in stops]   #Removing stopwords
    return( " ".join( meaningful_words )) #Joining the words

In [ ]:
for i in range(0,number_reviews_train):
    cleanHeadline = get_words(X_train[i]) #Processing the data and getting words with no special characters, numbers or html tags
    cleanHeadlines_train.append( cleanHeadline )

In [ ]:
for i in range(0,number_reviews_test):
    cleanHeadline = get_words(X_test[i]) #Processing the data and getting words with no special characters, numbers or html tags
    cleanHeadlines_test.append( cleanHeadline )

In [ ]:
#vectorize = CountVectorizer(stop_words='english')
vectorize = CountVectorizer(analyzer = "word",max_features = 1700)    #(stop_words='english')
bagOfWords_train = vectorize.fit_transform(cleanHeadlines_train)
X_train = bagOfWords_train.toarray()

In [ ]:
bagOfWords_test = vectorize.transform(cleanHeadlines_test)
X_test = bagOfWords_test.toarray()

### Auto ML

In [ ]:
num_folds = 10
seed = 7
scoring = 'accuracy'

models = []
models.append(('LR' , LogisticRegression()))
models.append(('LDA' , LinearDiscriminantAnalysis()))
#models.append(('KNN' , KNeighborsClassifier()))
#models.append(('CART' , DecisionTreeClassifier()))
#models.append(('NB' , GaussianNB()))
#models.append(('SVM' , SVC()))
models.append(('MNB', MultinomialNB()))
models.append(('RF' , RandomForestClassifier(n_estimators=50)))
#models.append(('XGBoost', XGBClassifier()))

In [ ]:
results = []
names = []

'''
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=42)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) '''

for name, model in models:
    clf = model
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    accu_score = accuracy_score(Y_test, Y_pred)
    print(name + ": " + str(accu_score))

## Save to CSV

In [ ]:
articleDataFrame.to_csv (runParams['output_file'][0], index = False, header=True)